# LLM Engine

In [1]:
!pip -q install --upgrade transformers accelerate bitsandbytes einops

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 48.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 11.2 MB/s eta 0:00:00


In [3]:
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TextStreamer

In [4]:
MODEL_ID = "Qwen/Qwen3-8B"

In [5]:
# Configuración de cuantización
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    use_fast=True,
    trust_remote_code=True
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.float16,
    trust_remote_code=True
)
model.eval()

#                                  -------------------------------
#           ---------             | --------                  --- |            -----------
# texto -> |Tokenizer| - > IDs -> ||Codebook| -> Vectores -> |LLM|| -> IDs -> |unTokenizer| -> Texto (output)
#           ---------             | --------                  --- |            -----------
#                                  -------------------------------

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/399 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Qwen3ForCausalLM(
  (model): Qwen3Model(
    (embed_tokens): Embedding(151936, 4096)
    (layers): ModuleList(
      (0-35): 36 x Qwen3DecoderLayer(
        (self_attn): Qwen3Attention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (q_norm): Qwen3RMSNorm((128,), eps=1e-06)
          (k_norm): Qwen3RMSNorm((128,), eps=1e-06)
        )
        (mlp): Qwen3MLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=12288, bias=False)
          (down_proj): Linear4bit(in_features=12288, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): Qwen3RMSNorm((4096,), eps=1e-06

In [12]:
def qwen( prompt: str,
          system: str = "You are a logical planning assitant",
          max_new_tokens: int=512) -> str:

    # Alta temperatura: Creativo (alucinar)
    #                   Las probabilidades seran homogeneas
    # Menor temperatura: Más determinista
    #                    Se aproxima a un one-hot

    # enable_thinking: Habilita el modo de pensamiento
    # do_sample: Muestre aleatorios entre tokens más probables
    # top-p: Los token más probables hasta tener p de probabilidad
    # top-k: Seleccionamos los k tokens mas probables
    # stream: Controla si los tokens se van generando en tiempo real

    messages = [
        {"role": "system", "content": system},
        {"role": "user", "content": prompt}
    ]

    # ID de tokens (respuesta)
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
        enable_thinking=False
    )
    # Traducimos los Ids como texto
    inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # Parámetros estrictos para tareas lógicas
    gen_kwargs = dict(
        max_new_tokens=max_new_tokens,
        do_sample=False,      # Apagar muestreo aleatorio para respuestas deterministas
    )

    with torch.no_grad():
        # resp = [input, output]
        out = model.generate(**inputs, **gen_kwargs)

    # Sólo la parte nueva:
    gen_ids = out[0, inputs.input_ids.shape[1]:]
    return tokenizer.decode(gen_ids, skip_special_tokens=True)

# Evaluator

In [13]:
def limpiar_accion(accion_texto):
    texto = accion_texto.replace('(', '').replace(')', '')
    return texto.strip().lower()

def calcular_score_plan(plan_generado, plan_optimo):
    P = [limpiar_accion(p) for p in plan_generado if p.strip()]
    G = [limpiar_accion(p) for p in plan_optimo if p.strip()]

    L_P = len(P)
    L_G = len(G)

    if L_P == 0:
        return 0.0

    score_horizonte = 2.0 if L_P == L_G else 0.0

    l_match = 0
    for p_accion, g_accion in zip(P, G):
        if p_accion == g_accion:
            l_match += 1
        else:
            break

    score_progreso = 3.0 * (l_match / L_G)
    score_exacto = 5.0 if (l_match == L_G and L_P == L_G) else 0.0

    return round(score_horizonte + score_progreso + score_exacto, 2)

# Student Agent

In [14]:
class AssemblyAgent:
    def __init__(self):
        self.system_prompt = "Eres un planificador experto en ensamblaje automatizado."

    def solve(self, scenario_context: str, llm_engine_func) -> list:
        """
        Recibe el texto del escenario y la funcion del motor LLM.
        Debe retornar una lista de strings con las acciones extraidas.
        """
        prompt_final = f"{scenario_context}\n\nAnaliza la situacion y dame el plan:"

        respuesta_bruta = llm_engine_func(
            prompt=prompt_final,
            system=self.system_prompt
        )

        lineas = respuesta_bruta.split('\n')
        acciones_parseadas = []
        for l in lineas:
            l_limpia = l.strip()
            if any(verbo in l_limpia for verbo in ["engage", "mount", "release", "dismount"]):
                acciones_parseadas.append(l_limpia)

        return acciones_parseadas

# Dev Test

In [16]:
import json

ARCHIVO_DESARROLLO = "Examples.json"

def dev_test(n_casos=10):
    print(f"Cargando dataset de desarrollo: {ARCHIVO_DESARROLLO}")
    with open(ARCHIVO_DESARROLLO, 'r') as f:
        casos = json.load(f)

    agente = AssemblyAgent()
    puntaje_total = 0.0
    casos_evaluados = min(n_casos, len(casos)) # Limite para pruebas rapidas

    print("-" * 50)
    for i in range(casos_evaluados):
        caso = casos[i]
        print(f"Evaluando Tarea ID: {caso['assembly_task_id']} (Longitud optima: {caso['complexity_level']})")

        plan_generado = agente.solve(caso['scenario_context'], qwen)
        plan_optimo = caso['target_action_sequence']

        # Calculo de metrica
        score = calcular_score_plan(plan_generado, plan_optimo)
        puntaje_total += score

        print(f"Plan Generado: {plan_generado}")
        print(f"Score obtenido: {score} / 10.0\n")

    promedio = puntaje_total / casos_evaluados
    print("-" * 50)
    print(f"Puntaje Promedio en Desarrollo: {round(promedio, 2)} / 10.0")

dev_test(10)

Cargando dataset de desarrollo: Examples.json
--------------------------------------------------
Evaluando Tarea ID: task_6a4ed4586d (Longitud optima: 4)
Plan Generado: []
Score obtenido: 0.0 / 10.0

Evaluando Tarea ID: task_b9a95877e2 (Longitud optima: 4)
Plan Generado: []
Score obtenido: 0.0 / 10.0

Evaluando Tarea ID: task_0d236ad4c6 (Longitud optima: 6)
Plan Generado: []
Score obtenido: 0.0 / 10.0

Evaluando Tarea ID: task_372c431054 (Longitud optima: 6)
Plan Generado: []
Score obtenido: 0.0 / 10.0

Evaluando Tarea ID: task_79c0ae73f7 (Longitud optima: 2)
Plan Generado: []
Score obtenido: 0.0 / 10.0

Evaluando Tarea ID: task_5cc798432f (Longitud optima: 4)
Plan Generado: []
Score obtenido: 0.0 / 10.0

Evaluando Tarea ID: task_78ac2d8d7f (Longitud optima: 6)
Plan Generado: []
Score obtenido: 0.0 / 10.0

Evaluando Tarea ID: task_9d7fd5f56c (Longitud optima: 6)
Plan Generado: []
Score obtenido: 0.0 / 10.0

Evaluando Tarea ID: task_4371c597dd (Longitud optima: 6)
Plan Generado: []
Scor

# Submit

In [17]:
import json

ARCHIVO_EVALUACION = "Task.json"
ARCHIVO_SALIDA = "submission.json"

def submit():
    print(f"Iniciando ejecucion sobre: {ARCHIVO_EVALUACION}")
    with open(ARCHIVO_EVALUACION, 'r') as f:
        casos = json.load(f)

    agente = AssemblyAgent()
    resultados_entrega = []

    for i, caso in enumerate(casos):
        task_id = caso['assembly_task_id']
        print(f"Procesando caso {i+1}/{len(casos)} (ID: {task_id})...")

        try:
            plan_generado = agente.solve(caso['scenario_context'], qwen)

            resultados_entrega.append({
                "assembly_task_id": task_id,
                "target_action_sequence": plan_generado
            })

        except Exception as e:
            print(f"ERROR critico en el caso {task_id}: {e}")
            print("Corrige tu codigo. https://www.youtube.com/watch?v=Y-U1calv6X8")
            return

    # Guardar
    with open(ARCHIVO_SALIDA, 'w') as f:
        json.dump(resultados_entrega, f, indent=4)

    print("-" * 50)
    print(f"Exito. Archivo '{ARCHIVO_SALIDA}' generado correctamente.")

submit()

Iniciando ejecucion sobre: Task.json
Procesando caso 1/50 (ID: task_f6c3f52f55)...
Procesando caso 2/50 (ID: task_07a18910c7)...
Procesando caso 3/50 (ID: task_cbe2649f6b)...
Procesando caso 4/50 (ID: task_4f181b1e7e)...
Procesando caso 5/50 (ID: task_9f39e7f413)...
Procesando caso 6/50 (ID: task_a51e02706c)...
Procesando caso 7/50 (ID: task_8a763f838b)...
Procesando caso 8/50 (ID: task_4d92dfa4d1)...
Procesando caso 9/50 (ID: task_19c005f4fa)...
Procesando caso 10/50 (ID: task_98c536f455)...
Procesando caso 11/50 (ID: task_b516b29d5b)...
Procesando caso 12/50 (ID: task_2833f3d973)...
Procesando caso 13/50 (ID: task_a2e1da433c)...
Procesando caso 14/50 (ID: task_b05ad833d2)...
Procesando caso 15/50 (ID: task_a90c8a54bb)...
Procesando caso 16/50 (ID: task_dae8652991)...
Procesando caso 17/50 (ID: task_07201ae0aa)...
Procesando caso 18/50 (ID: task_37dd853373)...
Procesando caso 19/50 (ID: task_784946134b)...
Procesando caso 20/50 (ID: task_32d7e53ce2)...
Procesando caso 21/50 (ID: task_